In [ ]:
import polars as pl
import numpy as np
import polars_ds as pld

In [ ]:
df = pl.DataFrame({
    "a": range(100),
    "y": [0] * 50 + [1] * 50
})
df.head()

In [ ]:
df.select(
    pl.col("y").num.woe(pl.col("a"))
)

In [ ]:
df = pl.DataFrame({
    "a": range(1000),
    "b": ["cat"] * 200 + ["dogs"] * 500 + ["lizards"] * 300,
    "y": np.random.randint(0, high = 2, size = 1000)
})
df.head()

In [ ]:
df = pl.DataFrame({
    "a": range(5000),
    "b": [np.sqrt(x) for x in range(5000)],
    "y": 0.1 * np.random.random(size=5000) + np.array(list(range(5000)))
})
df.head()

In [ ]:

df.select(
    pl.col("y").num.lstsq_report(
        pl.col("a"), pl.col("b"),
        add_bias = False
    ).alias("report")
).unnest("report")

In [ ]:
from statsmodels.api import OLS
target = df["y"].to_numpy()
data = df.select("a", "b").to_numpy()

In [ ]:
%%timeit
_ = OLS(target, data).fit().summary()

In [ ]:
df.select(
    pl.col("a1").num.psi(pl.col("a2"))
).item(0,0)

In [ ]:
df.select(
    pl.col("b").num.psi_discrete(pl.col("a"))
)

In [ ]:
df.select(
    pl.col("a").value_counts()
).unnest("a")

In [ ]:
pl.col("a").value_counts().name

In [ ]:
vc = pl.col("b").qcut(10, left_closed = False, allow_duplicates=True, include_breaks=True).struct.field("brk").value_counts().sort()
df.select(
    vc
)

In [ ]:
df.select(
    pl.col("b").num.psi(pl.col("c")),
    pl.col("b").min().alias("b_min"),
    pl.col("c").min().alias("c_min"),
)

In [ ]:
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 2,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

In [ ]:
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 2,
        threshold = 4, # <= threshold hamming distance away
        metric = "hamming"
    ).alias("similar_words_from_vocab"),
)

In [ ]:
df.select(
    pl.col("a").str2.similar_words(
        vocab = ["WORLD", "AAAAA", "ABCDEFG", "ZIV"],
        k = 2,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

In [ ]:
df = pl.DataFrame({
    "a": range(100_000),
    "b": range(100_000),
    "c": range(100_000),
})

In [ ]:
df = pl.DataFrame({
    "y": [[1,0,0],[0,1,0],[0,0,1],[1,0,0],[0,1,0]],
    "pred":[[0.1, 0.5, 0.4], [0.2, 0.6, 0.2], [0.4, 0.1, 0.5], [0.9, 0.05, 0.05], [0.2, 0.5, 0.3]]
})

In [ ]:
df.select(
    pl.col("pred").list.get(pl.col("y"))
)

In [ ]:
df.select(
    pl.col("y").metric.categorical_cross_entropy(pl.col("pred"), normalize=True, dense = False)
).item(0,0)